# Poisson Image Editing
Patrick Pérez, Michel Gangnet, Andrew Blake

Les noms

### Imports

In [ ]:
import numpy as np
import scipy as scp
import pylab as pyl
import pywt
import pandas as pd
import holoviews as hv
import param
import panel as pn
import matplotlib.pyplot as plt
from panel.pane import LaTeX
import requests
hv.extension('bokeh')
import warnings
warnings.filterwarnings('ignore')
from PIL import Image
from io import BytesIO

### Chargement des images

In [ ]:
caselist=['Ours','Femme piscine','Homme piscine']

In [ ]:
local=1
def chargeData(name):
    if local:
        if name=='Ours':
            target=Image.open("./data/target_ocean.jpg")
            source=Image.open("./data/source_ours.jpg")
            mask2=Image.open("./data/mask_ours.png")
            
        if name=='Femme piscine':
            target=Image.open("./data/target_ocean.jpg")
            source=Image.open("./data/source_femme_piscine.jpg")
            mask2=Image.open("./data/mask_femme_piscine.jpg")
            
        if name=='Homme piscine':
            target=Image.open("./data/target_ocean2.jpg")
            source=Image.open("./data/source_homme_piscine.jpg")
            mask2=Image.open("./data/mask_homme_piscine.jpg")
     
        source = source.resize(np.shape(target)[:2][::-1])
        mask2 = mask2.resize(np.shape(target)[:2][::-1])
            
        target = np.array(target).astype(float)
        source = np.array(source).astype(float)
        mask2 = np.array(mask2).astype(float)/255
    return target,source,mask2

In [ ]:
target,source,mask2=chargeData('Homme piscine')



optionsRGB=dict(width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
optionsGray=dict(cmap='gray',width=300,height=300,xaxis=None,yaxis=None,toolbar=None)
pn.Row(hv.RGB(target.astype('uint8')).opts(**optionsRGB),hv.RGB(source.astype('uint8')).opts(**optionsRGB),hv.Image((mask2*255).astype('uint8')).opts(**optionsGray))

### Calcul des gradients

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return y
def Div(y):
    x=DivHor(y[0])+DivVer(y[1])
    return x

### Gradient et projection

In [ ]:
def Proj(im,ma,iref):
    #si x y pas dans le masque
    #si appartient au masque, pas de contrainte, u(x,y)
    res = im*ma + iref*(1-ma)
    
    return res

def GradientFonc(x,y):
    #Gradient de la fonctionnelle
    #Gradient de la fonction pas evident du tout lol ijbol mdr
    #f(x) = 1/2 y = grad S et A = grad ||Ax-y|| // (grad x - grad s)
    #gradf(x) = AT (Ax-y) 
    
    g = Gradient(x)
    r1 = g[0]-y[0]
    r2 = g[1]-y[1]
    
    res = DivHor(r1)+DivVer(r2)
    
    return res

### Naive fusion with a simple projection

In [ ]:
proj = Proj(source[:,:,0],mask2[:,:,0],target[:,:,0])
grad = GradientFonc(target[:,:,0],Gradient(source[:,:,0]))

hv.Image((proj).astype('uint8')).opts(**optionsGray)

In [ ]:
# Divide into 3 channels

target0=target[:,:,0]
source0=source[:,:,0]
target1=target[:,:,1]
source1=source[:,:,1]
target2=target[:,:,2]
source2=source[:,:,2]

In [ ]:
# Fusion

projRGB = np.zeros((np.shape(target)))
projRGB[:,:,0]= Proj(source0,mask2[:,:,0],target0)
projRGB[:,:,1]= Proj(source1,mask2[:,:,0],target1)
projRGB[:,:,2]= Proj(source2,mask2[:,:,0],target2)

hv.RGB((projRGB).astype('uint8')).opts(**optionsRGB)

### Forward Backward Poisson (à changer !!!!!)

Computes the projected gradient on a grayscale image (single color chanel). The function returns the last iterate of the sequence and a curve of the values of iterates.

In [ ]:
def FBPoissonEditing2(targ,sour,ma,step,Niter):
    
    f = []
    GradS = Gradient(sour)
    x = targ
    
    for i in range(Niter):
        
        x = Proj(x-step*GradientFonc(x,GradS),ma,targ)
        
        g = Gradient(x)
    
        cout = (np.linalg.norm(g[0]-GradS[0],ord='fro') + np.linalg.norm(g[1]-GradS[1],ord='fro'))/2# /!\ norme 2 matrice Forb
        f.append(cout)
        
    return np.clip(x,0,255),f[10:]

In [ ]:
step=1/4
niter=265
res0,f0=FBPoissonEditing2(target0,source0,mask2[:,:,0],step,niter)
res1,f1=FBPoissonEditing2(target1,source1,mask2[:,:,0],step,niter)
res2,f2=FBPoissonEditing2(target2,source2,mask2[:,:,0],step,niter)
res=target.copy()
res[:,:,0]=res0
res[:,:,1]=res1
res[:,:,2]=res2

hv.RGB(res.astype('uint8')).opts(**optionsRGB)

In [ ]:
hv.Curve(f0+f1+f2).opts(xaxis=None,toolbar=None)